In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [2]:
import importlib
importlib.reload(dh)
importlib.reload(md)

<module 'model' from '/home/mikhail/Education/Semester_11/TransportNet/Stable Dynamic & Beckman/model.py'>

In [3]:
import numba
numba.__version__

'0.53.1'

In [4]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)
from platform import python_version
print('python', python_version())
print('numpy', np.__version__)
print('pandas', pd.__version__)
import graph_tool
print('graph_tool', graph_tool.__version__)

/usr/bin/python3
3.6.9 (default, Jan 26 2021, 15:33:00) 
[GCC 8.4.0]
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
python 3.6.9
numpy 1.19.5
pandas 1.1.4
graph_tool 2.43 (commit 9d41331e, Wed Jul 7 15:32:52 2021 +0200)


# Beckmann model paradox
$\rho = 1$ 
$\mu = 1$

In [5]:
beckmann_paradox_save = 'beckmann_paradox_results/'

In [6]:
net_name = 'BeckmanParadox_net.tntp'
trips_name = 'BeckmanParadox_trips.tntp'

MU = 1
RHO = 1

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

no_tax_model = md.Model(graph_data, graph_correspondences, 
                        total_od_flow, mu = MU, rho = RHO)

tax_model = md.ModelWithTaxes(graph_data, graph_correspondences, 
                              total_od_flow, mu = MU, rho = RHO)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,1.000000e-04,0.000001
1,1,True,3,True,1.000000e+20,45.000000
2,2,True,4,True,1.000000e+20,45.000000
3,3,True,4,True,1.000000e-04,0.000001
4,2,True,3,True,1.000000e+20,0.000001


In [7]:
max_iter = 100000

solver_kwargs = {'eps_abs': 0.3,
                 'max_iter': max_iter, 'stop_crit': 'dual_gap',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}

print('No tax calculation')
tic = time.time()
no_tax_result = no_tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

print('Tax calculation')
tic = time.time()
tax_result = tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

No tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 55 sec
Tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 56 sec


In [8]:
def form_list(arr):
     return ["{0:0.2f}".format(el) for el in arr]

def anarchy_cost(tax_result, no_tax_result):
    tax_vals = tax_result['taxes']
    tax_t = tax_result['times'] - tax_vals
    tax_f = tax_result['flows']
    no_tax_t = no_tax_result['times']
    no_tax_f = no_tax_result['flows']
    
    tax_total = np.dot(tax_t, tax_f)
    no_tax_total = np.dot(no_tax_t, no_tax_f)
    print("Tax flows:\t\t", *form_list(tax_f))
    print("Taxes costs:\t\t", *form_list(tax_vals))
    print("Tax total time:\t\t{:.2f}\n".format(tax_total))
    print("No tax flows:\t\t", *form_list(no_tax_f))
    print("No tax total time:\t{:.2f}".format(no_tax_total))
    print("\nAnarchy price:\t\t", no_tax_total / tax_total)

anarchy_cost(tax_result, no_tax_result)

Tax flows:		 2263.01 1736.99 1736.95 2263.05 526.07
Taxes costs:		 22.63 0.00 0.00 22.63 0.00
Tax total time:		257573.65

No tax flows:		 4000.00 -0.00 -0.00 4000.00 4000.00
No tax total time:	317840.27

Anarchy price:		 1.2339782250795903


# Stable dynamics paradox
$\rho = 1$ 
$\mu = 0$

In [9]:
beckmann_paradox_save = 'stable_paradox_results/'

In [10]:
net_name = 'StableParadox_net.tntp'
trips_name = 'StableParadox_trips.tntp'

MU = 0
RHO = 1

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

no_tax_model = md.Model(graph_data, graph_correspondences, 
                        total_od_flow, mu = MU, rho = RHO)

tax_model = md.ModelWithTaxes(graph_data, graph_correspondences, 
                              total_od_flow, mu = MU, rho = RHO)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,True,2,True,2000.0,15.0
1,1,True,3,True,2000.0,60.0
2,2,True,3,True,2000.0,30.0


In [11]:
max_iter = 100000

solver_kwargs = {'eps_abs': 0.3,
                 'max_iter': max_iter, 'stop_crit': 'dual_gap',
                 'verbose' : False, 'verbose_step': 2000, 'save_history' : True}

print('No tax calculation')
tic = time.time()
no_tax_result = no_tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs,
                                              base_flows = 0.8 * graph_data['graph_table']['capacity'])
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

print('Tax calculation')
tic = time.time()
tax_result = tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs,
                                        base_flows = 0.8 * graph_data['graph_table']['capacity'])
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

No tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 136 sec
Tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
	Elapsed time: 140 sec


In [12]:
anarchy_cost(tax_result, no_tax_result)

Tax flows:		 501.75 998.25 2001.75
Taxes costs:		 0.00 0.00 15.00
Tax total time:		127473.80

No tax flows:		 501.75 998.25 2001.75
No tax total time:	157500.83

Anarchy price:		 1.2355545421277998


In [13]:
net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

MU = 0.25
RHO = 0.15

handler = dh.DataHandler()
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)

no_tax_model = md.Model(graph_data, graph_correspondences, 
                        total_od_flow, mu = MU, rho = RHO)

tax_model = md.ModelWithTaxes(graph_data, graph_correspondences, 
                              total_od_flow, mu = MU, rho = RHO)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,False,117,True,9000.0,1.090458
1,2,False,87,True,9000.0,1.090458
2,3,False,74,True,9000.0,1.090458
3,4,False,233,True,9000.0,1.090458
4,5,False,165,True,9000.0,1.090458


In [14]:
max_iter = 10000

solver_kwargs = {'eps_abs': 0.3,
                 'max_iter': max_iter, 'stop_crit': 'dual_gap',
                 'verbose' : True, 'verbose_step': 2000, 'save_history' : True}

print('No tax calculation')
tic = time.time()
no_tax_result = no_tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

print('Tax calculation')
tic = time.time()
tax_result = tax_model.find_equilibrium(solver_name = 'wda', solver_kwargs = solver_kwargs)
toc = time.time()
print('\tElapsed time: {:.0f} sec'.format(toc - tic))

No tax calculation
Composite optimization...
Oracles created...
Weighted dual averages method...
Primal_init = 1.29606e+06
Dual_init = -1.24813e+06
Duality_gap_init = 47933.4

Iterations number: 2000
Primal_func_value = 1.28604e+06
Dual_func_value = -1.286e+06
Duality_gap = 34.197
Duality_gap / Duality_gap_init = 0.000713428

Iterations number: 4000
Primal_func_value = 1.28604e+06
Dual_func_value = -1.28602e+06
Duality_gap = 18.1014
Duality_gap / Duality_gap_init = 0.000377636

Iterations number: 6000
Primal_func_value = 1.28603e+06
Dual_func_value = -1.28602e+06
Duality_gap = 12.4101
Duality_gap / Duality_gap_init = 0.000258903

Iterations number: 8000
Primal_func_value = 1.28603e+06
Dual_func_value = -1.28602e+06
Duality_gap = 9.47457
Duality_gap / Duality_gap_init = 0.000197661

Iterations number: 10000
Primal_func_value = 1.28603e+06
Dual_func_value = -1.28603e+06
Duality_gap = 7.67665
Duality_gap / Duality_gap_init = 0.000160152

Result: iterations number exceeded
Total iters: 100

In [15]:
anarchy_cost(tax_result, no_tax_result)

Tax flows:		 7074.90 9662.50 7669.00 12173.80 2586.80 6576.60 7137.10 722.10 1576.15 661.35 72.20 77.10 485.80 488.20 37.00 125.20 407.10 249.00 648.30 2185.80 683.00 746.90 291.10 503.60 1134.45 1507.35 624.81 899.59 1522.50 191.00 184.90 4583.36 3970.84 2158.75 816.25 314.20 233.50 1460.97 622.23 626.85 517.95 1373.31 1562.19 1563.50 2075.30 1413.08 644.82 824.71 958.49 3909.69 1412.51 1529.20 435.30 646.17 286.63 222.90 114.70 884.53 627.27 71.16 27.10 42.50 16.20 355.28 142.09 614.13 359.91 17.40 19.90 538.17 248.48 -0.00 -0.00 347.90 931.42 258.57 -0.00 718.04 17.40 126.97 282.29 833.36 376.28 190.78 -0.00 2790.68 594.02 1381.15 923.40 1632.72 181.23 1842.17 4080.93 1813.95 1287.78 1908.85 3786.90 2007.49 944.09 2146.80 3003.48 13602.20 13602.20 7021.08 168.21 7189.28 7189.28 5612.96 -0.00 5612.96 4904.19 731.82 5636.01 5481.20 1179.87 6661.07 2266.20 5402.80 7669.00 5676.60 5676.60 2192.23 2143.36 4335.59 4204.16 1214.50 5418.66 4182.72 213.17 4395.90 3925.63 942.10 4867.73 4867.